In [ ]:
import whisperx
import json
import torch
import os
from dotenv import load_dotenv
from pyannote.audio import Pipeline


In [ ]:
load_dotenv()
SECRET_KEY = os.getenv("HF_KEY")

In [8]:
audio_file2 = r"C:\Users\tejat\Desktop\Assignment\Feature1\audio_file.wav"

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisperx.load_model("medium", device,compute_type="float32")

No language specified, language will be first be detected for each audio file (increases inference time).
>>Performing voice activity detection using Pyannote...


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\tejat\anaconda3\envs\assignment\lib\site-packages\whisperx\assets\pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.7.1+cpu. Bad things might happen unless you revert torch to 1.x.


In [9]:
transcription_result = model.transcribe(audio_file2)

Detected language: en (1.00) in first 30s of audio...


In [10]:
align_model, metadata = whisperx.load_align_model(language_code=transcription_result["language"], device=device)
transcription_result = whisperx.align(transcription_result["segments"], align_model, metadata, audio_file2, device)

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to C:\Users\tejat/.cache\torch\hub\checkpoints\wav2vec2_fairseq_base_ls960_asr_ls960.pth


100%|██████████| 360M/360M [01:39<00:00, 3.78MB/s] 


In [ ]:
pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  use_auth_token=SECRET_KEY)
diarization = pipeline(r"audio_file.wav", num_speakers=3)

c:\Users\tejat\anaconda3\envs\assignment\lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)


In [ ]:
def assign_speakers_to_segments(transcription_segments, diarization_result):
    result = []
    for segment in transcription_segments:
        segment_start = segment['start']
        segment_end = segment['end']
        segment_text = segment['text']
        speaker = None
        max_overlap = 0
        for turn, _, speaker_label in diarization_result.itertracks(yield_label=True):
            overlap_start = max(segment_start, turn.start)
            overlap_end = min(segment_end, turn.end)
            overlap_duration = max(0, overlap_end - overlap_start)
            if overlap_duration > max_overlap:
                max_overlap = overlap_duration
                speaker = speaker_label
        result.append({
            "speaker": speaker,
            "text": segment_text.strip(),
            "start": segment_start,
            "end": segment_end
        })
    return result



In [ ]:

final_result = assign_speakers_to_segments(transcription_result["segments"], diarization)
output_json = json.dumps(final_result, indent=2, ensure_ascii=False)
print(output_json)

[
  {
    "speaker": "SPEAKER_02",
    "text": "Good afternoon, Professor Nelson.",
    "start": 0.554,
    "end": 2.616
  },
  {
    "speaker": "SPEAKER_02",
    "text": "How are you doing?",
    "start": 2.776,
    "end": 3.557
  },
  {
    "speaker": "SPEAKER_00",
    "text": "Good afternoon, Jennifer.",
    "start": 4.858,
    "end": 6.34
  },
  {
    "speaker": "SPEAKER_00",
    "text": "I am doing well, and you?",
    "start": 6.66,
    "end": 8.141
  },
  {
    "speaker": "SPEAKER_02",
    "text": "I'm great, thank you.",
    "start": 8.161,
    "end": 11.064
  },
  {
    "speaker": "SPEAKER_02",
    "text": "This is my friend Lucy.",
    "start": 11.565,
    "end": 12.886
  },
  {
    "speaker": "SPEAKER_02",
    "text": "She is thinking about applying to this college.",
    "start": 13.627,
    "end": 15.849
  },
  {
    "speaker": "SPEAKER_02",
    "text": "She has a few questions.",
    "start": 16.369,
    "end": 17.751
  },
  {
    "speaker": "SPEAKER_02",
    "text": "Wou